In [1]:
#import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#create a set of random latitude and longitude combinations
lats = np.random.uniform(low = -90, high = 90, size = 1500)
lngs = np.random.uniform(low = -180, high = 180, size = 1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
coordinates = list(lat_lngs)

In [4]:
from citipy import citipy

In [5]:
# create a list for holding the cities
cities = []
#identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name

    #if city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# print the city count to confirm suficiient count
len(cities)

581

In [6]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [7]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key 
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=871fd5a894c8e293d2e3ef714b59dbe1


In [8]:
#import the datatime module from the datetime library
from datetime import datetime

In [13]:
#create an empty list to hold the weather data
city_data = []
#print the beginning of the logging
print("Beginning Data Retrieval   ")
print("---------------------------")

#create counters
record_count = 1
set_count = 1

# loop thru all the cities in our list
for i, city in enumerate(cities):

    #group cities in sets of 50 for logging purposes
    if (i% 50 == 0 and i>=50):
        set_count+=1
        record_count = 1
    #create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")

    #log the URL, record, and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    #add 1 to the record count
    record_count+=1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({
            
            "City": city.title(),
            "Lat": city_lat,
            "Lng": city_lng,
            "Max Temp": city_max_temp,
            "Humidity": city_humidity,
            "Cloudiness": city_clouds,
            "Wind Speed": city_wind,
            "Country": city_country,
            "Date": city_date })

#if an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
    #indicate that Data Loading is complete.
    print("------------------------  ")
    print("Data Retrieval Complete   ")
    print("------------------------  ")


lete   
------------------------  
Processing Record 22 of Set 9 | asyut
------------------------  
Data Retrieval Complete   
------------------------  
Processing Record 23 of Set 9 | minudasht
------------------------  
Data Retrieval Complete   
------------------------  
Processing Record 24 of Set 9 | garden city
------------------------  
Data Retrieval Complete   
------------------------  
Processing Record 25 of Set 9 | falealupo
City not found. Skipping...
------------------------  
Data Retrieval Complete   
------------------------  
Processing Record 26 of Set 9 | langres
------------------------  
Data Retrieval Complete   
------------------------  
Processing Record 27 of Set 9 | huilong
------------------------  
Data Retrieval Complete   
------------------------  
Processing Record 28 of Set 9 | truth or consequences
------------------------  
Data Retrieval Complete   
------------------------  
Processing Record 29 of Set 9 | altay
------------------------  
Data 

In [14]:
len(city_data)

538

In [19]:
# convert the array of dictionaries to a pandas df
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,79.63,64,19,6.71,PF,2021-02-01 03:59:38
1,Punta Arenas,-53.1500,-70.9167,53.60,62,90,29.93,CL,2021-02-01 03:59:38
2,Saint-Philippe,-21.3585,55.7679,84.20,51,0,8.05,RE,2021-02-01 03:59:38
3,Berlevag,70.8578,29.0864,15.80,86,0,10.36,NO,2021-02-01 03:59:38
4,Kodiak,57.7900,-152.4072,30.20,64,1,5.75,US,2021-02-01 03:59:38
5,Ajacuba,20.0875,-99.1125,49.73,84,9,4.90,MX,2021-02-01 03:59:39
6,Sarahan,31.5167,77.8000,43.77,49,9,5.66,IN,2021-02-01 03:59:39
7,Bardiyah,31.7561,25.0865,61.03,43,0,18.88,LY,2021-02-01 03:59:39
8,Longyearbyen,78.2186,15.6401,6.80,33,75,12.66,SJ,2021-02-01 04:01:17
9,Ushtobe,45.2514,77.9838,27.73,88,100,2.95,KZ,2021-02-01 04:02:19


In [21]:
#create the output file csv
output_data_file  = "weather_data/cities.csv"
#export the city_data into a CSV
city_data_df.to_csv(output_data_file, index_label="City_ID")